# Bibliotecas

In [1]:
from obterDados import obterSimboloPosicao, obterSimboloData
import numpy as np
import tensorflow as tf
import gymnasium as gym
import gym_anytrading
from stable_baselines3 import DQN
from json import dumps

c:\Users\aurel\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Obtendo dados de M5 e D1

In [2]:
rawDataM5 = obterSimboloPosicao()[:-100*108]
rawDataM5

,open,high,low,close,tick_volume,real_volume
time,,,,,,
2020-02-06 18:00:00,4288.0,4289.0,4287.0,4288.5,937,5308
2020-02-06 18:05:00,4289.0,4290.0,4287.5,4290.0,921,6269
2020-02-06 18:10:00,4290.0,4291.5,4288.0,4288.0,1162,8771
2020-02-07 09:00:00,4300.5,4306.0,4300.5,4302.5,6673,29348
2020-02-07 09:05:00,4302.5,4303.5,4298.0,4299.0,9860,34027
...,...,...,...,...,...,...
2023-05-17 17:35:00,4947.0,4947.0,4943.5,4944.5,1681,5619
2023-05-17 17:40:00,4944.5,4946.0,4944.0,4945.0,1850,6588
2023-05-17 17:45:00,4945.5,4948.0,4944.5,4948.0,2055,6785


In [3]:
rawDataM5.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
rawDataM5

,Open,High,Low,Close,tick_volume,Volume
time,,,,,,
2020-02-06 18:00:00,4288.0,4289.0,4287.0,4288.5,937,5308
2020-02-06 18:05:00,4289.0,4290.0,4287.5,4290.0,921,6269
2020-02-06 18:10:00,4290.0,4291.5,4288.0,4288.0,1162,8771
2020-02-07 09:00:00,4300.5,4306.0,4300.5,4302.5,6673,29348
2020-02-07 09:05:00,4302.5,4303.5,4298.0,4299.0,9860,34027
...,...,...,...,...,...,...
2023-05-17 17:35:00,4947.0,4947.0,4943.5,4944.5,1681,5619
2023-05-17 17:40:00,4944.5,4946.0,4944.0,4945.0,1850,6588
2023-05-17 17:45:00,4945.5,4948.0,4944.5,4948.0,2055,6785


In [4]:
rawDataD1 = obterSimboloData(rawDataM5.index[-1], n=int(len(rawDataM5)/108)+200)
rawDataD1

,open,high,low,close,tick_volume,real_volume
time,,,,,,
2019-04-03,3841.5,3889.0,3841.0,3876.5,352969,1108529
2019-04-04,3877.0,3895.0,3855.0,3860.5,373990,1180951
2019-04-05,3863.0,3884.0,3849.5,3879.0,327693,1025388
2019-04-08,3874.0,3883.5,3848.5,3855.5,306660,1004395
2019-04-09,3859.0,3870.0,3852.5,3856.0,300248,1000182
...,...,...,...,...,...,...
2023-05-11,4977.0,5004.5,4946.0,4952.0,833486,2925054
2023-05-12,4953.0,4975.0,4924.5,4939.5,745650,2644096
2023-05-15,4938.0,4949.0,4902.0,4910.5,772038,2640870


In [5]:
rawDataD1.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
rawDataD1

,Open,High,Low,Close,tick_volume,Volume
time,,,,,,
2019-04-03,3841.5,3889.0,3841.0,3876.5,352969,1108529
2019-04-04,3877.0,3895.0,3855.0,3860.5,373990,1180951
2019-04-05,3863.0,3884.0,3849.5,3879.0,327693,1025388
2019-04-08,3874.0,3883.5,3848.5,3855.5,306660,1004395
2019-04-09,3859.0,3870.0,3852.5,3856.0,300248,1000182
...,...,...,...,...,...,...
2023-05-11,4977.0,5004.5,4946.0,4952.0,833486,2925054
2023-05-12,4953.0,4975.0,4924.5,4939.5,745650,2644096
2023-05-15,4938.0,4949.0,4902.0,4910.5,772038,2640870


# Criando ambiente e testando aleatoriamente

In [6]:
from customTradingEnv2 import CustomTradingEnv

env = CustomTradingEnv(df_5min=rawDataM5, df_daily=rawDataD1)

In [7]:
# observation = env.reset()
# done = False
# cont = 0
# while not done:
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)

# info

# Treinando modelo com aprendizado por reforço

In [10]:
from stable_baselines3.common.callbacks import CheckpointCallback

modelo = DQN('MultiInputPolicy', env, verbose=1, tensorboard_log='logs/dqn/')

models_dir = 'models/DQN'
checkpointCallback = CheckpointCallback(save_freq=10000, save_path=models_dir)
modelo.learn(total_timesteps=len(rawDataM5)*200, callback=checkpointCallback)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/dqn/DQN_1


c:\Users\aurel\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\buffers.py:590: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 12.90GB > 2.43GB
  warnings.warn(


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.36e+04 |
|    ep_rew_mean      | 826      |
|    exploration_rate | 0.929    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 353      |
|    time_elapsed     | 380      |
|    total_timesteps  | 134210   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 8.13     |
|    n_updates        | 21052    |
----------------------------------


### Continuar treinamento a partir de modelo salvo

In [ ]:
# from stable_baselines3.common.callbacks import CheckpointCallback

# models_dir = 'models/DQN'
# model_path = f"{models_dir}/rl_model_10340000_steps.zip"
# modelo = A2C.load(model_path, env=env)
# checkpointCallback = CheckpointCallback(save_freq=10000, save_path=models_dir)
# modelo.learn(total_timesteps=len(rawDataM5)*300, callback=checkpointCallback, reset_num_timesteps=False)

# Testando modelo treinado

In [ ]:
testDataM5 = obterSimboloPosicao()[-(20*108+300):]
testDataM5.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)
testDataD1 = obterSimboloData(testDataM5.index[-1], n=int(len(testDataM5)/108)+200)
testDataD1.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'real_volume': 'Volume'
}, inplace=True)

In [ ]:
from customTradingEnv2 import CustomTradingEnv

env = CustomTradingEnv(df_5min=testDataM5, df_daily=testDataD1)

In [ ]:
models_dir = 'models/DQN'
model_path = f"{models_dir}/rl_model_30000_steps.zip"
modelo = DQN.load(model_path, env=env)

In [ ]:
observation = env.reset()
done = False
while not done:
    action = modelo.predict(observation)
    observation, reward, done, info = env.step(int(action[0]))
    # env.render()

info